<a href="https://colab.research.google.com/github/ankit071105/ML_Projects/blob/main/Snake-Game/Snake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python mediapipe numpy

In [ ]:
import cv2
import numpy as np
import math
import random
import mediapipe as mp
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, HTML
from base64 import b64decode
import time

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [ ]:
class SnakeGame:
    def __init__(self, screen_width=640, screen_height=480):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.reset_game()
        self.snake_color = (0, 255, 0)
        self.food_color = (0, 0, 255)
        self.head_color = (0, 200, 200)
        self.text_color = (255, 255, 255)

    def reset_game(self):
        self.points = []
        self.lengths = []
        self.current_length = 0
        self.allowed_length = 150
        self.previous_head = self.screen_width//2, self.screen_height//2
        self.score = 0
        self.game_over = False
        self.food_position = self.generate_food()

    def generate_food(self):
        margin = 50
        return random.randint(margin, self.screen_width-margin), random.randint(margin, self.screen_height-margin)

    def update(self, img, current_head):
        if self.game_over:
            self.display_game_over(img)
            return img

        px, py = self.previous_head
        cx, cy = current_head

        self.points.append([cx, cy])
        distance = math.hypot(cx - px, cy - py)
        self.lengths.append(distance)
        self.current_length += distance
        self.previous_head = cx, cy

        if self.current_length > self.allowed_length:
            while self.current_length > self.allowed_length and len(self.lengths) > 0:
                self.current_length -= self.lengths[0]
                self.lengths.pop(0)
                self.points.pop(0)

        fx, fy = self.food_position
        if abs(cx - fx) < 20 and abs(cy - fy) < 20:
            self.allowed_length += 50
            self.score += 1
            self.food_position = self.generate_food()

        if len(self.points) > 1:
            for i, point in enumerate(self.points[:-1]):
                cv2.line(img, tuple(point), tuple(self.points[i+1]), self.snake_color, 10)

        cv2.circle(img, (cx, cy), 15, self.head_color, cv2.FILLED)
        cv2.circle(img, self.food_position, 10, self.food_color, cv2.FILLED)

        if len(self.points) > 5:
            pts = np.array(self.points[:-20], np.int32)
            pts = pts.reshape((-1, 1, 2))
            if cv2.pointPolygonTest(pts, (cx, cy), False) > 0:
                self.game_over = True

        return img

    def display_game_over(self, img):
        cv2.putText(img, "GAME OVER",
                   (self.screen_width//2-150, self.screen_height//2-50),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.putText(img, f"Final Score: {self.score}",
                   (self.screen_width//2-120, self.screen_height//2+10),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, self.text_color, 2)
        cv2.putText(img, "Press R to Restart",
                   (self.screen_width//2-120, self.screen_height//2+60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, self.text_color, 2)


In [ ]:
js = Javascript('''
async function setupWebcam() {
  const div = document.createElement('div');
  const video = document.createElement('video');
  video.setAttribute('id', 'webcam');
  video.style.display = 'block';
  video.width = 640;
  video.height = 480;
  div.appendChild(video);
  document.body.appendChild(div);

  const stream = await navigator.mediaDevices.getUserMedia({video: true});
  video.srcObject = stream;
  await video.play();
  return true;
}

async function captureFrame() {
  const video = document.getElementById('webcam');
  const canvas = document.createElement('canvas');
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  canvas.getContext('2d').drawImage(video, 0, 0);
  return canvas.toDataURL('image/jpeg', 0.8);
}
''')

display(js)
display(HTML('<style>video { transform: scaleX(-1); }</style>'))
def main():
    print("Starting Snake Game with Hand Control...")
    from google.colab.output import eval_js
    eval_js('setupWebcam()')

    width, height = 640, 480
    game = SnakeGame(width, height)

    try:
        while True:
            frame_data = eval_js('captureFrame()')
            frame_bytes = b64decode(frame_data.split(',')[1])
            frame_np = np.frombuffer(frame_bytes, dtype=np.uint8)
            img = cv2.imdecode(frame_np, flags=1)
            img = cv2.flip(img, 1)
            img = cv2.resize(img, (width, height))

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    lm = hand_landmarks.landmark[8]
                    cx, cy = int(lm.x * width), int(lm.y * height)
                    mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    img = game.update(img, (cx, cy))
            else:
                img = game.update(img, game.previous_head)

            # Add black board on top for score
            cv2.rectangle(img, (0,0), (width,40), (0,0,0), -1)
            cv2.putText(img, f"Score: {game.score}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

            cv2_imshow(img)

            if game.game_over:
                print("Game Over! Final Score:", game.score)
                restart = input("Press 'r' and Enter to restart, or any other key to quit: ")
                if restart.lower() == 'r':
                    game.reset_game()
                    continue
                else:
                    break
            time.sleep(0.03)  # Smoother refresh rate

    except Exception as e:
        print("Error:", e)
    finally:
        print("Game closed.")

if __name__ == "__main__":
    main()